<a href="https://colab.research.google.com/github/jherna85-pixel/SDS-510/blob/Module-5/Module_5_Project_Text_Analysis_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SDS 510: Module 5 Project
## Text Analysis Basics

Jennifer Hernandez

Import what is needed and load in dataset

In [ ]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score


In [ ]:
df = pd.read_json('jeopardy.json')

In [ ]:
df.head()

,category,air_date,question,value,answer,round,show_number
0,HISTORY,2004-12-31,"'For the last 8 years of his life, Galileo was...",$200,Copernicus,Jeopardy!,4680
1,ESPN's TOP 10 ALL-TIME ATHLETES,2004-12-31,'No. 2: 1912 Olympian; football star at Carlis...,$200,Jim Thorpe,Jeopardy!,4680
2,EVERYBODY TALKS ABOUT IT...,2004-12-31,'The city of Yuma in this state has a record a...,$200,Arizona,Jeopardy!,4680
3,THE COMPANY LINE,2004-12-31,"'In 1963, live on ""The Art Linkletter Show"", t...",$200,McDonald\'s,Jeopardy!,4680
4,EPITAPHS & TRIBUTES,2004-12-31,"'Signer of the Dec. of Indep., framer of the C...",$200,John Adams,Jeopardy!,4680


In [ ]:
print(df['value'].value_counts())

value
$400       42244
$800       31860
$200       30455
$600       20377
$1000      19539
           ...  
$4,238         1
$16,400        1
$1,347         1
$2547          1
$11,200        1
Name: count, Length: 149, dtype: int64


Clean up text to ensure program can read it. Clean up questions (to recognize patterns) and values (to integers)


1.   change all to lowercase
2.   remove puncuation
3.   remove extra white space
4.   value to integer



In [ ]:
def clean_question (question):
  question = str(question).lower()
  question = re.sub(r'[^a-z]',' ',question)
  question = re.sub (r'\s+',' ', question)
  return question.strip ()

df['clean_question'] = df['question'].apply(clean_question)


In [ ]:
def clean_value(value):
  if pd.isna(value):
    return None
  value = str(value).replace('$','').replace(',','')
  if value.isdigit():
    return int(value)
  else:
        return None

df ['clean_value']= df['value'].apply(clean_value)
df = df.dropna(subset=['clean_value'])


Calculate median and add new label for high and low values

* '> 600(median), 1, high value'
* '< 600(median), 0, low value'



In [ ]:
median_value=df['clean_value'].median()
median_value

600.0

In [ ]:
df['high_low'] = (df['clean_value'] > median_value).astype(int)

Vectorize text, train, and test to see if the words in the question give us a high or low value

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.clean_question, df.high_low, random_state=1)

In [ ]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_tf = tfidf_vectorizer.fit_transform(X_train)
X_test_tf = tfidf_vectorizer.transform(X_test)


In [ ]:
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_tf, y_train)
predictions = naive_bayes.predict(X_test_tf)

In [ ]:
print('Accuracy: ', accuracy_score(y_test,predictions))

Accuracy:  0.5853086790188283
